### Task-2 Taxi Problem
2. Read and complete the following tutorial: https://www.learndatasci.com/tutorials/reinforcement-q- learning-scratch-python-openai-gym/, ensuring that you can reproduce all steps discussed.

 - **Design Solution of Taxi Problem is implemented below using Python & Q-Learning technique**

In [44]:
# Importing required python & Opengym packages
import numpy as np
import gym
import matplotlib.pyplot as plt


In [45]:
pip install --upgrade pip

  Using cached pip-21.1.2-py3-none-any.whl (1.5 MB)
  Attempting uninstall: pip
    Found existing installation: pip 21.0.1
    Uninstalling pip-21.0.1:
Note: you may need to restart the kernel to use updated packages.


ERROR: Could not install packages due to an OSError: [WinError 5] Access is denied: 'c:\\programdata\\anaconda3\\lib\\site-packages\\pip-21.0.1.dist-info\\entry_points.txt'
Consider using the `--user` option or check the permissions.

You should consider upgrading via the 'C:\ProgramData\Anaconda3\python.exe -m pip install --upgrade pip' command.


In [46]:
import gym

env = gym.make("Taxi-v3").env

env.render()

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+



In [47]:
env.reset() # reset environment to a new, random state
env.render()

print("Action Space {}".format(env.action_space))
print("State Space {}".format(env.observation_space))

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+

Action Space Discrete(6)
State Space Discrete(500)


In [48]:
state = env.encode(3, 1, 2, 0) # (taxi row, taxi column, passenger index, destination index)
print("State:", state)

env.s = state
env.render()

State: 328
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+



In [49]:
env.P[328]

{0: [(1.0, 428, -1, False)],
 1: [(1.0, 228, -1, False)],
 2: [(1.0, 348, -1, False)],
 3: [(1.0, 328, -1, False)],
 4: [(1.0, 328, -10, False)],
 5: [(1.0, 328, -10, False)]}

### Without Reinforcement Learning

In [50]:
env.s = 328  # set environment to illustration's state

epochs = 0
penalties, reward = 0, 0

frames = [] # for animation

done = False

while not done:
    action = env.action_space.sample()
    state, reward, done, info = env.step(action)

    if reward == -10:
        penalties += 1
    
    # Put each rendered frame into dict for animation
    frames.append({
        'frame': env.render(mode='ansi'),
        'state': state,
        'action': action,
        'reward': reward
        }
    )

    epochs += 1
    
    
print("Timesteps taken: {}".format(epochs))
print("Penalties incurred: {}".format(penalties))
print("Rewards incurred: {}".format(reward))
#print(frames)

Timesteps taken: 2324
Penalties incurred: 764
Rewards incurred: 20


In [51]:
from IPython.display import clear_output
from time import sleep

def print_frames(frames):
    for i, frame in enumerate(frames):
        clear_output(wait=True)
        print(frame['frame'])
        print(f"Timestep: {i + 1}")
        print(f"State: {frame['state']}")
        print(f"Action: {frame['action']}")
        print(f"Reward: {frame['reward']}")
        sleep(.1)
print(type(frames))        
print_frames(frames)


+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (Dropoff)

Timestep: 2324
State: 0
Action: 5
Reward: 20


#### The agent is not performing well. 

The agent is taking thousands of timesteps and making lots of wrong drop offs to deliver just one passenger to the right destination.This is because agent is not learning from past experience. This is not going to optimize the result even uf we run this over and over.   

***The agent has no memory to store the best action for each state, which is exactly what Reinforcement Learning will do for us.***

#### Let's implement the problem with Reinforcement Learning using Q-Algorithm

  -  QLearning enable agent to learn from the reward action of the enviroenment and the best action is taken for a specific state.
  -  In this Taxi problem we will store the reward value in Q Table
  

#### Training the Agent
  -  Initialize the Q-table to a 500×6 matrix of zeros:

In [52]:
import numpy as np
q_table = np.zeros([env.observation_space.n, env.action_space.n])

 - Create the training algorithm that will update this Q-table as the agent explores the environment over thousands of episodes.

  -  First part of while not done, deciding whether to pick a random action or to exploit the already computed Q-values. This is done simply by using the epsilon value and comparing it to the random.uniform(0, 1) function, which returns an arbitrary number between 0 and 1.

  -  Execute the chosen action in the environment to obtain the next_state and the reward from performing the action. 
  
  - After that, calculate the maximum Q-value for the actions corresponding to the next_state, and with that, we can easily update our Q-value to the new_q_value:

In [53]:
%%time
"""Training the agent"""

import random
from IPython.display import clear_output

# Hyperparameters
alpha = 0.1
gamma = 0.6
epsilon = 0.1

# For plotting metrics
all_epochs = []
all_penalties = []

for i in range(1, 100001):
    state = env.reset()

    epochs, penalties, reward, = 0, 0, 0
    done = False
    
    while not done:
        if random.uniform(0, 1) < epsilon:
            action = env.action_space.sample() # Explore action space
        else:
            action = np.argmax(q_table[state]) # Exploit learned values

        next_state, reward, done, info = env.step(action) 
        
        old_value = q_table[state, action]
        next_max = np.max(q_table[next_state])
        
        new_value = (1 - alpha) * old_value + alpha * (reward + gamma * next_max)
        q_table[state, action] = new_value

        if reward == -10:
            penalties += 1

        state = next_state
        epochs += 1
        
    if i % 100 == 0:
        clear_output(wait=True)
        print(f"Episode: {i}")

print("Training finished.\n")

Episode: 100000
Training finished.

Wall time: 1min 23s


In [54]:
#The q value stored in q tables
print(q_table[328])

[ -2.40475931  -2.27325184  -2.4119682   -2.35748025 -10.81498341
 -10.02554342]


### Evaluating Agent
  -  Next action is selected using the Q-value table

In [57]:
"""Evaluate agent's performance after Q-learning"""

total_epochs, total_penalties, total_reward = 0, 0, 0
episodes = 100

for _ in range(episodes):
    state = env.reset()
    epochs, penalties, reward = 0, 0, 0
    
    done = False
    
    while not done:
        action = np.argmax(q_table[state])
        state, reward, done, info = env.step(action)

        if reward == -10:
            penalties += 1

        epochs += 1

    total_penalties += penalties
    total_reward += reward
    total_epochs += epochs

print(f"Results after {episodes} episodes:")
print(f"Average timesteps per episode: {total_epochs / episodes}")
print(f"Average penalties per episode: {total_penalties / episodes}")
print(f"Average reward per episode: {total_reward / episodes}")

Results after 100 episodes:
Average timesteps per episode: 13.04
Average penalties per episode: 0.0
Average reward per episode: 20.0


  -  The results shows significant imprvement in learning process, agent has not incurred any penalty.
  -  Our agent Taxi has peformed well with 100 passengers and performed pickup/drop action perfectly.
  